In [1]:
from FASTSim_Package.envs.FASTSim_RL_Environment import *
from utils import *   # import replay buffer for experienced replay and plotting function
from DQN_Agent import *
import numpy as np
import matplotlib.pyplot as plt
from matplotlib import style

style.use('ggplot')

In [2]:
if __name__ == '__main__':
    env = gym.make('FASTSim-v0')
    n_games = 500
    agent = Agent(gamma=0.99, lr=0.0005, input_dims=8, n_actions=4, batch_size=64,
                  epsilon=1.0, eps_dec=0.9999995, eps_min=0.0, mem_size=1000000)
        
    scores = []
    eps_history = []

    for i in range(n_games):
        done = False
        score = 0
        state = env.reset()
        while not done:
            action = agent.choose_action(state)
            state_, reward, done, info = env.step(action)
            score += reward
            agent.remember(state, action, reward, state_, done) # store experience
            observation = observation_   # new state -- state
            agent.learn()   
            
        eps_history.append(agent.epsilon)
        scores.append(score)

        avg_score = np.mean(scores[max(0, i-100):(i+1)])
        print('episode: ', i,'score: %.2f' % score,
              ' average score %.2f' % avg_score)

        if i % 10 == 0 and i > 0:
            agent.save_model()

    filename = 'FASTSim.png'

    x = [i+1 for i in range(n_games)]
    plotLearning(x, scores, eps_history, filename)

1370 1370 1370 1370 1370 1370
action: 2


ValueError: The truth value of an array with more than one element is ambiguous. Use a.any() or a.all()